<a href="https://colab.research.google.com/github/titsitits/UNamur_Python_Analytics/blob/master/4_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mickaël Tits
CETIC
mickael.tits@cetic.be

# Un exemple concret: un peu nutrition et de Natural Language Processing (NLP)
Dans cet exemple, nous avons une collection de denrées alimentaires, et quelques informations sur ces aliments. Chaque aliment a le même type de propriété (un nom, un poids, des valeurs énergétiques).

In [5]:



#extract singular versions of words in text (defaults plurals and exceptions are used, but you can override default rules)
def singulize(string, pluriels = None, exceptions = None):
  
  """
  Extract singular versions of words in text (defaults plurals and exceptions are used, but you can override default rules)
  Default plurals: {'ois':'ois','s':'','eaux':'eau','aux':'al','x':''}
  Default exceptions: {'os':'os','chacals':'chacal','souris':'souris','rabais':'rabais','prix':'prix', 'taux':'taux','rhinoceros':'rhinoceros','jus':'jus','noix':'noix','mais':'mais'}

  """

  #singulize (french ad-hoc method...)
  #règles du pluriel
  if pluriels is None:
      pluriels = {'ois':'ois','s':'','eaux':'eau','aux':'al','x':''}
  #exceptions
  if exceptions is None:
      exceptions = {'os':'os','chacals':'chacal','souris':'souris','rabais':'rabais','prix':'prix', 'taux':'taux','rhinoceros':'rhinoceros','jus':'jus','noix':'noix','mais':'mais', 'chips':'chips'}

  #Une fonction dans une fonction! Pourquoi diable ? Elle n'est pas appelée ailleurs que dans la fonction singulize. Par soucis de clarté, on la déclare donc uniquement dans la portée de cette fonction.
  def singulize_word(word):

      if word in exceptions:

          return exceptions[word]

      isplural = [word.endswith(k) for k in pluriels]

      if any(isplural):

          #take first key
          keyid = isplural.index(True)
          key = list(pluriels)[keyid]

          #to replace last occurence, reverse all strings and replace first occurrence
          return word[::-1].replace(key[::-1], pluriels[key][::-1], 1)[::-1]

      return word

  return ' '.join([singulize_word(word) for word in string.split()])



#remove stopwords from text (defaults stopwords are used but you can override default rules)
def remove_stopwords(string, stopwords = None):
      
  """
  Remove stopwords from text (defaults stopwords are used but you can override default rules)
  Default stopwords: ['de','du','le','les','aux','la','des', 'a', 'une', 'un', 'au','d','l']
  """
  #remove stopwords
  if stopwords is None:
      stopwords = ['de','du','le','les','aux','la','des', 'a', 'à', 'une', 'un', 'au','d','l','et']

  string = string.replace("d'","")
  string = string.replace("l'","")        
  return ' '.join([word for word in string.split() if word not in (stopwords)])



#Un petit test
sentence1 = "cake à la banane noix de cajou"
sentence2 = "cakes aux bananes et aux noix de cajou"

s1 = singulize( remove_stopwords(sentence1) )
s2 = singulize( remove_stopwords(sentence2) )



print(s1)
print(s2)

if s1 == s2:
  print("Les phrases sont identiques")
  

  


cake banane noix cajou
cake banane noix cajou
Les phrases sont identiques


In [4]:
# Pour éviter de redéfinir à chaque fois toutes les règles de traitement (pluriels, stopwords, exceptions) quand on appelle une fonction, on peut à la place créer une classe et définir les règles comme des attributs de cette classe.


class NLP:
  
  def __init__(self, plurals, stopwords, exceptions):
    
    self.pluriels = plurals
    self.stopwords = stopwords
    self.exceptions = exceptions
    
  def singulize(self, string):
    
    """
    Extract singular versions of words in text
    """    
    
    #Une fonction dans une fonction! Pourquoi diable ? Elle n'est pas appelée ailleurs que dans la fonction singulize. Par soucis de clarté, on la déclare donc uniquement dans la portée de cette fonction.
    def singulize_word(word):

        if word in self.exceptions:

            return self.exceptions[word]

        isplural = [word.endswith(k) for k in self.pluriels]

        if any(isplural):

            #take first key
            keyid = isplural.index(True)
            key = list(self.pluriels)[keyid]

            #to replace last occurence, reverse all strings and replace first occurrence
            return word[::-1].replace(key[::-1], self.pluriels[key][::-1], 1)[::-1]

        return word

    return ' '.join([singulize_word(word) for word in string.split()])
  
  def remove_stopwords(self, string):

    """
    Remove stopwords from text
    """

    string = string.replace("d'","")
    string = string.replace("l'","")        
    return ' '.join([word for word in string.split() if word not in (stopwords)])
  
  def simplify(self, string):
    
    string = self.remove_stopwords(string)
    return self.singulize(string)


#Définition unique des règles  
pluriels = {'ois':'ois','s':'','eaux':'eau','aux':'al','x':''}
exceptions = {'os':'os','chacals':'chacal','souris':'souris','rabais':'rabais','prix':'prix', 'taux':'taux','rhinoceros':'rhinoceros','jus':'jus','noix':'noix','mais':'mais', 'chips':'chips'}
stopwords = ['de','du','le','les','aux','la','des', 'a', 'à', 'une', 'un', 'au','d','l','et']

textprocessor = NLP(pluriels, stopwords, exceptions)

print(textprocessor.pluriels)


#Test (le résultat devrait être le même)
s1 = textprocessor.simplify(sentence1)
s2 = textprocessor.simplify(sentence2)

print(s1)
print(s2)

if s1 == s2:
  print("Les phrases sont identiques")

{'ois': 'ois', 's': '', 'eaux': 'eau', 'aux': 'al', 'x': ''}
cake banane noix cajou
cake banane noix cajou
Les phrases sont identiques


In [0]:
class Food:
  
  def __init__(self, nom, poids, calories_per_100g):
    
    self.nom = nom
    self.poids = poids
    self.cal = calories_per_100g
    
    self.simplify_name()

  def simplify_name(self):
    
    self.nom = singulize( remove_stopwords(self.nom) )
    self.nom = self.nom.capitalize()
    
  def totcal(self):
    
    return self.poids*self.cal/100
  
  def display(self):
    
    print("%s: %d cal/100g, %d g, calories totales: %d" % (self.nom, self.cal, self.poids, self.totcal()) )

In [0]:
#Un inventaire d'aliments

dataset = {"noms":["des pommes","chips",sentence1, sentence2, "pomme"], 
           "poids": [150, 120, 75, 80, 145], 
           "calories_per_100g":[52, 536, 320, 310, 53]}

In [16]:


n = len(dataset["noms"])

#Je peut utiliser la classe Food pour représenter mes aliments de manière structurée:
aliments = [Food(dataset["noms"][i], dataset["poids"][i], dataset["calories_per_100g"][i]) for i in range(0,n)]

for item in aliments:
  item.display()

Pomme: 52 cal/100g, 150 g, calories totales: 78
Chips: 536 cal/100g, 120 g, calories totales: 643
Cake banane noix cajou: 320 cal/100g, 75 g, calories totales: 240
Cake banane noix cajou: 310 cal/100g, 80 g, calories totales: 248
Pomme: 53 cal/100g, 145 g, calories totales: 76


In [17]:
#Lors de la création d'un aliment, son nom est déjà traité (voir la méthode __init__). La liste contient donc deux aliments qui ont le même nom: "Pomme"
#retirons les duplicatas
#On peut pour cela passer par un dictionnaire: chaque clé doit être unique. On utilise donc le nom de l'aliment comme clé:
aliments_dict = {item.nom:item for item in aliments}
unique_aliments = list(aliments_dict.values())

for item in unique_aliments:
  item.display()


def find_lightest(foods):
  
  """
  Arguments d'entrée:
  foods: liste d'aliments (objets de la classe Food)
  
  Résultats:
  un tuple contanent le meilleur aliment, et son indice dans la liste
  """

  allcals = [item.totcal() for item in aliments]
  mincals = min(allcals)
  best = allcals.index(mincals)
  
  return (aliments[best], best)

meilleur_aliment = find_lightest(aliments)[0]

print("Aliment le plus léger:", meilleur_aliment.nom)

Pomme: 53 cal/100g, 145 g, calories totales: 76
Chips: 536 cal/100g, 120 g, calories totales: 643
Cake banane noix cajou: 310 cal/100g, 80 g, calories totales: 248
Aliment le plus léger: Pomme


In [0]:
#corbeille...



#pomme = Food("des pommes", poids = 150, calories_per_100g = 52)
#paquet_chips = Food("chips", poids = 120, calories_per_100g = 536)
#cake = Food(sentence1, poids = 75, calories_per_100g = 320)
#another_cake = Food(sentence2, poids = 80, calories_per_100g = 310)
#pomme_dup = Food("pomme", poids = 145, calories_per_100g = 53)
#aliments = [pomme, paquet_chips, cake, another_cake, pomme_dup]